In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv('../.env')

allData = pd.read_csv('https://drive.deta.sh/v1/b0x22rtxtdf/data/files/download?name=allData.csv', storage_options={'X-API-Key':os.environ.get("DETA_PROJECT_KEY")})
allData.ds = pd.to_datetime(allData.ds)
allData

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
0,2021-01-01 01:00:00,1,1,1,1,2,2,0,0,0,...,0,0,0.0,0.0,1,0,0,0,3,3
1,2021-01-01 02:00:00,2,3,1,2,3,5,1,1,0,...,2,2,0.0,0.0,1,1,0,0,3,3
2,2021-01-01 03:00:00,0,3,1,3,1,6,0,1,0,...,2,1,0.0,0.0,0,0,0,0,3,3
3,2021-01-01 04:00:00,1,4,0,3,1,7,1,2,0,...,2,0,0.0,0.0,0,0,0,0,4,3
4,2021-01-01 05:00:00,1,5,1,4,2,9,1,3,0,...,1,1,0.0,0.0,0,0,0,0,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26348,2023-11-29 12:00:00,8,51,8,47,16,98,2,16,0,...,13,6,2.0,0.0,7,1,0,3,20,19
26349,2023-11-29 12:00:00,10,55,6,43,16,98,3,17,0,...,13,6,2.0,0.0,7,1,0,3,20,19
26350,2023-11-29 13:00:00,11,69,4,44,15,113,1,18,0,...,10,6,1.0,0.0,13,0,2,3,20,19
26351,2023-11-29 13:00:00,10,65,5,48,15,113,1,18,0,...,10,6,1.0,0.0,13,0,2,3,20,19


In [2]:
df = allData.copy()
df.isna().sum().sum()
df.ds = pd.to_datetime(df.ds)
df.tail(5)

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
26348,2023-11-29 12:00:00,8,51,8,47,16,98,2,16,0,...,13,6,2.0,0.0,7,1,0,3,20,19
26349,2023-11-29 12:00:00,10,55,6,43,16,98,3,17,0,...,13,6,2.0,0.0,7,1,0,3,20,19
26350,2023-11-29 13:00:00,11,69,4,44,15,113,1,18,0,...,10,6,1.0,0.0,13,0,2,3,20,19
26351,2023-11-29 13:00:00,10,65,5,48,15,113,1,18,0,...,10,6,1.0,0.0,13,0,2,3,20,19
26352,2023-11-29 14:00:00,15,84,9,53,24,137,6,24,0,...,10,7,3.0,0.0,10,0,1,5,21,18


In [1]:
from prophet import Prophet
from tqdm import tqdm

In [4]:
output = pd.DataFrame()
first_run = True

for column in tqdm(df.columns.to_list()):
    if column in ['ds']:
        continue
    try:
        print('working on '+column)

        m = Prophet(interval_width=0.95)
        m.fit(df[['ds', column]].rename(columns={column:'y'}))
        future = m.make_future_dataframe(periods=24*1, freq='H')
        forecast = m.predict(future.tail(24*14))
        if first_run:
            output['ds'] = forecast['ds']
        
        for forecast_column in ['yhat', 'yhat_lower', 'yhat_upper']:
            kwargs = {column+'_'+forecast_column : forecast[forecast_column]}
            output = output.assign(**kwargs)   
        first_run = False
    except:
        print(column + ' failed')



  0%|          | 0/46 [00:00<?, ?it/s]

working on INFLOW_STRETCHER


14:14:44 - cmdstanpy - INFO - Chain [1] start processing
14:14:49 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 2/46 [00:06<02:22,  3.23s/it]

working on Infl_Stretcher_cum


14:14:50 - cmdstanpy - INFO - Chain [1] start processing
14:14:56 - cmdstanpy - INFO - Chain [1] done processing
  7%|▋         | 3/46 [00:13<03:32,  4.95s/it]

...

14:21:06 - cmdstanpy - INFO - Chain [1] start processing
14:21:11 - cmdstanpy - INFO - Chain [1] done processing
 96%|█████████▌| 44/46 [06:28<00:17,  8.74s/it]

working on PSYCH1


14:21:13 - cmdstanpy - INFO - Chain [1] start processing
14:21:24 - cmdstanpy - INFO - Chain [1] done processing
 98%|█████████▊| 45/46 [06:41<00:09,  9.97s/it]

working on PSYCH_WAITINGADM


14:21:26 - cmdstanpy - INFO - Chain [1] start processing
14:21:39 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 46/46 [06:56<00:00,  9.06s/it]


In [5]:
output

,ds,INFLOW_STRETCHER_yhat,INFLOW_STRETCHER_yhat_lower,INFLOW_STRETCHER_yhat_upper,Infl_Stretcher_cum_yhat,Infl_Stretcher_cum_yhat_lower,Infl_Stretcher_cum_yhat_upper,INFLOW_AMBULATORY_yhat,INFLOW_AMBULATORY_yhat_lower,INFLOW_AMBULATORY_yhat_upper,...,RAZ_XRAY_MORE2H_yhat_upper,RAZ_CT_MORE2H1_yhat,RAZ_CT_MORE2H1_yhat_lower,RAZ_CT_MORE2H1_yhat_upper,PSYCH1_yhat,PSYCH1_yhat_lower,PSYCH1_yhat_upper,PSYCH_WAITINGADM_yhat,PSYCH_WAITINGADM_yhat_lower,PSYCH_WAITINGADM_yhat_upper
0,2023-11-16 15:00:00,9.372677,4.761938,14.195761,86.678985,53.625166,121.496760,6.670304,1.973523,11.585410,...,3.415547,2.464569,-0.688219,5.434393,13.205138,7.049230,19.792659,8.965866,3.219119,14.581424
1,2023-11-16 16:00:00,8.624814,3.801546,13.593240,99.627985,62.437326,133.960454,6.334271,1.660312,11.145248,...,3.106638,2.469529,-0.556165,5.575368,13.125717,6.810233,19.245163,8.823037,3.510200,14.380197
2,2023-11-16 17:00:00,7.861612,3.093843,12.883545,106.732350,72.134901,141.502217,5.832562,0.793958,10.894051,...,3.175348,2.452457,-0.653105,5.791903,13.034825,6.421819,19.432305,8.704350,3.172089,14.257442
3,2023-11-16 18:00:00,7.254323,2.671563,11.522382,107.177992,72.823259,143.381621,5.246529,0.063297,10.038163,...,3.189028,2.503606,-0.599309,5.622773,12.949044,6.086552,19.175051,8.622259,2.426632,14.398532
4,2023-11-16 19:00:00,6.882685,2.204374,11.946306,107.987108,72.902946,145.550080,4.755868,0.143230,9.605422,...,3.246857,2.612539,-0.383126,5.692024,12.908222,5.998879,19.808633,8.605364,2.657486,14.075948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2023-11-30 10:00:00,8.519662,3.797807,13.431115,45.770872,10.370415,78.546544,7.767328,3.248775,11.914920,...,2.841791,1.328970,-1.905159,4.162326,16.677761,10.292310,22.948094,12.878918,7.038739,18.353225
332,2023-11-30 11:00:00,9.675963,4.678008,14.265940,58.321414,22.574938,92.654653,8.250726,3.659296,12.799600,...,2.948915,1.381354,-1.638001,4.298229,16.636992,10.054212,23.153321,12.720312,6.826799,18.234088
333,2023-11-30 12:00:00,10.272874,5.267760,15.065222,65.753625,32.779085,99.816730,7.897408,3.061340,12.780339,...,2.725915,1.615495,-1.486409,4.596350,16.483918,10.018385,23.079304,12.496575,6.946639,18.059560
334,2023-11-30 13:00:00,10.365968,5.290833,14.844847,70.007922,36.695670,106.158461,7.263926,2.595969,11.814880,...,3.142372,1.950820,-1.036957,4.865009,16.298498,9.348941,22.652183,12.253595,6.622481,18.257814


In [6]:
from deta import Deta
from dotenv import load_dotenv
import os

load_dotenv()

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))

drive = deta.Drive("data")

In [7]:
output.to_csv('anomaly_detection_ranges.csv', index=False)

drive.put(name='anomaly_detection_ranges.csv', path='anomaly_detection_ranges.csv')

'anomaly_detection_ranges.csv'

In [3]:
import pandas as pd

In [11]:
daily_inflow = pd.read_csv('https://www.dropbox.com/scl/fi/e61kmtweg90tq5hpqrnsf/daily_inflow.csv?rlkey=15r15klj0s0005fvhvlt2iuqt&raw=1')
daily_inflow.ds = pd.to_datetime(daily_inflow.ds)

column = 'Daily_Inflow_Total'
m = Prophet(interval_width=0.95, weekly_seasonality=True, yearly_seasonality=True, daily_seasonality=False, seasonality_mode='additive', changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, holidays_prior_scale=10.0)
m.add_country_holidays(country_name='CA')
m.fit(daily_inflow[['ds', column]].rename(columns={column: 'y'}))
future = m.make_future_dataframe(periods=30, freq='D')
forecast = m.predict(future.tail(30+14))

22:42:50 - cmdstanpy - INFO - Chain [1] start processing
22:42:50 - cmdstanpy - INFO - Chain [1] done processing


In [6]:
daily_inflow

,ds,Daily_Inflow_Total
0,2021-01-01,138
1,2021-01-02,161
2,2021-01-03,169
3,2021-01-04,219
4,2021-01-05,212
...,...,...
1751,2025-10-18,237
1752,2025-10-19,233
1753,2025-10-20,308
1754,2025-10-21,290


In [7]:
future

,ds
0,2021-01-01
1,2021-01-02
2,2021-01-03
3,2021-01-04
4,2021-01-05
...,...
1781,2025-11-17
1782,2025-11-18
1783,2025-11-19
1784,2025-11-20


In [12]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2025-10-09,262.663896,219.964644,306.524705
1,2025-10-10,267.568367,223.604406,305.996929
2,2025-10-11,233.514556,188.080949,275.610741
3,2025-10-12,222.169860,177.183458,263.301319
4,2025-10-13,275.493074,233.819133,321.541833
5,2025-10-14,274.983801,232.877021,318.338065
6,2025-10-15,266.661825,223.425212,311.062990
7,2025-10-16,262.455543,216.824501,307.477337
8,2025-10-17,267.100442,222.690701,311.757946
9,2025-10-18,232.733406,189.604536,274.677373
